# Credit Card Fraud – Anomaly Detection & Explainability

This notebook contains the data preparation steps performed on our dataset.

## Loading the data

In [ ]:
%pip -q install -U pandas pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 13.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.1 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.1 which is incompatible.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.1 which is incompatible.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.


In [ ]:
from pathlib import Path
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')
from pathlib import Path

# General Settings
cards = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/MA/datasets/archive/accessories/sd254_cards.csv")
users = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/MA/datasets/archive/accessories/sd254_users.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Datetime Feature Engineering & Categorical Encoding

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/MA/datasets/archive//credit_card_transactions-ibm_v2_2008_prepped.csv")
df.head()

,Unnamed: 0,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?,trans_date_trans_time,trans_hour,day_of_week,category
0,0,0,0,2008,1,1,06:34,$90.39,Swipe Transaction,4060646732831064559,La Verne,CA,91750.0,5411,NaN,No,2008-01-01 06:34:00,6,Tuesday,Retail Outlet Services
1,1,0,0,2008,1,1,13:06,$66.00,Swipe Transaction,2027553650310142703,Mira Loma,CA,91752.0,5541,NaN,No,2008-01-01 13:06:00,13,Tuesday,Retail Outlet Services
2,2,0,0,2008,1,1,13:14,$-66.00,Swipe Transaction,2027553650310142703,Mira Loma,CA,91752.0,5541,NaN,No,2008-01-01 13:14:00,13,Tuesday,Retail Outlet Services
3,3,0,0,2008,1,1,13:15,$43.06,Swipe Transaction,2027553650310142703,Mira Loma,CA,91752.0,5541,NaN,No,2008-01-01 13:15:00,13,Tuesday,Retail Outlet Services
4,4,0,0,2008,1,1,13:19,$10.22,Swipe Transaction,-5023497618971072366,Mira Loma,CA,91752.0,5812,NaN,No,2008-01-01 13:19:00,13,Tuesday,Miscellaneous Stores


In [ ]:
print(f"The dataset has {df.shape[0]} rows and {df.shape[1]} columns.")

The dataset has 1223460 rows and 20 columns.


## Merging Datasets

In [ ]:
print("cards columns:", cards.columns.tolist())
print("users columns:", users.columns.tolist())
print("df columns:", df.columns.tolist())

cards columns: ['User', 'CARD INDEX', 'Card Brand', 'Card Type', 'Card Number', 'Expires', 'CVV', 'Has Chip', 'Cards Issued', 'Credit Limit', 'Acct Open Date', 'Year PIN last Changed', 'Card on Dark Web']
users columns: ['Person', 'Current Age', 'Retirement Age', 'Birth Year', 'Birth Month', 'Gender', 'Address', 'Apartment', 'City', 'State', 'Zipcode', 'Latitude', 'Longitude', 'Per Capita Income - Zipcode', 'Yearly Income - Person', 'Total Debt', 'FICO Score', 'Num Credit Cards']
df columns: ['Unnamed: 0', 'User', 'Card', 'Year', 'Month', 'Day', 'Time', 'Amount', 'Use Chip', 'Merchant Name', 'Merchant City', 'Merchant State', 'Zip', 'MCC', 'Errors?', 'Is Fraud?', 'trans_date_trans_time', 'trans_hour', 'day_of_week', 'category']


In [ ]:
users.head()

,Person,Current Age,Retirement Age,Birth Year,Birth Month,Gender,Address,Apartment,City,State,Zipcode,Latitude,Longitude,Per Capita Income - Zipcode,Yearly Income - Person,Total Debt,FICO Score,Num Credit Cards
0,Hazel Robinson,53,66,1966,11,Female,462 Rose Lane,NaN,La Verne,CA,91750,34.15,-117.76,$29278,$59696,$127613,787,5
1,Sasha Sadr,53,68,1966,12,Female,3606 Federal Boulevard,NaN,Little Neck,NY,11363,40.76,-73.74,$37891,$77254,$191349,701,5
2,Saanvi Lee,81,67,1938,11,Female,766 Third Drive,NaN,West Covina,CA,91792,34.02,-117.89,$22681,$33483,$196,698,5
3,Everlee Clark,63,63,1957,1,Female,3 Madison Street,NaN,New York,NY,10069,40.71,-73.99,$163145,$249925,$202328,722,4
4,Kyle Peterson,43,70,1976,9,Male,9620 Valley Stream Drive,NaN,San Francisco,CA,94117,37.76,-122.44,$53797,$109687,$183855,675,1


In [ ]:
cards.head()

,User,CARD INDEX,Card Brand,Card Type,Card Number,Expires,CVV,Has Chip,Cards Issued,Credit Limit,Acct Open Date,Year PIN last Changed,Card on Dark Web
0,0,0,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
1,0,1,Visa,Debit,4956965974959986,12/2020,393,YES,2,$21968,04/2014,2014,No
2,0,2,Visa,Debit,4582313478255491,02/2024,719,YES,2,$46414,07/2003,2004,No
3,0,3,Visa,Credit,4879494103069057,08/2024,693,NO,1,$12400,01/2003,2012,No
4,0,4,Mastercard,Debit (Prepaid),5722874738736011,03/2009,75,YES,1,$28,09/2008,2009,No


In [ ]:
user_df = df['User']

print("Unique values in User from df:", user_df.unique())
print("Value counts:\n", user_df.value_counts())

Unique values in User from df: [   0    1    2 ... 1995 1996 1997]
Value counts:
 User
1249    4590
262     4416
332     4261
1101    4225
1150    3886
        ... 
92         1
347        1
70         1
992        1
1054       1
Name: count, Length: 1219, dtype: int64


In [ ]:
import duckdb

con = duckdb.connect(database=':memory:')

# Cards table
con.execute("""
CREATE TABLE cards AS
SELECT
  "User",
  "CARD INDEX",
  "Card Brand",
  "Card Type",
  "Expires",
  "Credit Limit",
  "Acct Open Date",
  "Card on Dark Web"
FROM read_csv_auto('/content/drive/MyDrive/Colab Notebooks/MA/datasets/archive/accessories/sd254_cards.csv')
""")

# Users table
con.execute("""
CREATE TABLE users AS
SELECT
  row_number() OVER () - 1 AS "User",
  "Current Age",
  "Gender",
  "Yearly Income - Person",
  "FICO Score",
  "Num Credit Cards"
FROM read_csv_auto('/content/drive/MyDrive/Colab Notebooks/MA/datasets/archive/accessories/sd254_users.csv')
""")

# Transactions table
con.execute("""
CREATE TABLE trans AS
SELECT
  * FROM read_csv_auto('/content/drive/MyDrive/Colab Notebooks/MA/datasets/archive//credit_card_transactions-ibm_v2_2008_prepped.csv')
""")

con.execute("""
COPY (
  SELECT
    t.*,
    c."Card Brand",
    c."Card Type",
    c."Expires",
    c."Credit Limit",
    c."Acct Open Date",
    c."Card on Dark Web",
    u."Current Age",
    u."Gender",
    u."Yearly Income - Person",
    u."FICO Score",
    u."Num Credit Cards"
  FROM trans AS t
  LEFT JOIN cards AS c ON t.User = c.User AND t.Card = c."CARD INDEX"
  LEFT JOIN users AS u ON t.User = u.User
)
TO '/content/drive/MyDrive/Colab Notebooks/MA/datasets/archive//credit_card_transactions_ibm_v2_2008_prepped_merged.csv'
(
  FORMAT CSV,
  HEADER TRUE,
  DELIMITER ','
);
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
df_merged = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/MA/datasets/archive//credit_card_transactions_ibm_v2_2008_prepped_merged.csv")
df_merged.head()

,column00,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,...,Card Type,Expires,Credit Limit,Acct Open Date,Card on Dark Web,Current Age,Gender,Yearly Income - Person,FICO Score,Num Credit Cards
0,0,0,0,2008,1,1,06:34:00,$90.39,Swipe Transaction,4060646732831064559,...,Debit,12/2022,$24295,09/2002,False,53,Female,$59696,787,5
1,1,0,0,2008,1,1,13:06:00,$66.00,Swipe Transaction,2027553650310142703,...,Debit,12/2022,$24295,09/2002,False,53,Female,$59696,787,5
2,2,0,0,2008,1,1,13:14:00,$-66.00,Swipe Transaction,2027553650310142703,...,Debit,12/2022,$24295,09/2002,False,53,Female,$59696,787,5
3,3,0,0,2008,1,1,13:15:00,$43.06,Swipe Transaction,2027553650310142703,...,Debit,12/2022,$24295,09/2002,False,53,Female,$59696,787,5
4,4,0,0,2008,1,1,13:19:00,$10.22,Swipe Transaction,-5023497618971072366,...,Debit,12/2022,$24295,09/2002,False,53,Female,$59696,787,5


In [ ]:
print(f"The dataset has {df_merged.shape[0]} rows and {df_merged.shape[1]} columns.")

The dataset has 1223460 rows and 31 columns.


## Handling Missing Values

In [ ]:
df = df_merged

In [ ]:
missing_counts = df.isnull().sum()
print("Missing values per column:\n", missing_counts)

# Check percentage of missing values
missing_percentage = (df.isnull().sum() / len(df)) * 100
print("\nPercentage of missing values:\n", missing_percentage)

Missing values per column:
 column00                        0
User                            0
Card                            0
Year                            0
Month                           0
Day                             0
Time                            0
Amount                          0
Use Chip                        0
Merchant Name                   0
Merchant City                   0
Merchant State             127392
Zip                        134917
MCC                             0
Errors?                   1204022
Is Fraud?                       0
trans_date_trans_time           0
trans_hour                      0
day_of_week                     0
category                        0
Card Brand                      0
Card Type                       0
Expires                         0
Credit Limit                    0
Acct Open Date                  0
Card on Dark Web                0
Current Age                     0
Gender                          0
Yearly Income - Pers

In [ ]:
errors_non_null = df['Errors?'].dropna()
print("Unique values in Errors?:", errors_non_null.unique())
print("Value counts:\n", errors_non_null.value_counts())

Unique values in Errors?: ['Insufficient Balance' 'Bad PIN' 'Bad Card Number' 'Technical Glitch'
 'Bad Expiration' 'Bad CVV' 'Bad Expiration,Bad CVV' 'Bad Zipcode'
 'Bad Card Number,Insufficient Balance'
 'Insufficient Balance,Technical Glitch' 'Bad PIN,Insufficient Balance'
 'Bad Card Number,Bad CVV' 'Bad PIN,Technical Glitch'
 'Bad CVV,Insufficient Balance' 'Bad Expiration,Technical Glitch'
 'Bad Card Number,Technical Glitch' 'Bad Zipcode,Insufficient Balance'
 'Bad Expiration,Insufficient Balance' 'Bad Zipcode,Technical Glitch']
Value counts:
 Errors?
Insufficient Balance                     12264
Bad PIN                                   2884
Technical Glitch                          2436
Bad Card Number                            610
Bad CVV                                    541
Bad Expiration                             511
Bad Zipcode                                 96
Insufficient Balance,Technical Glitch       32
Bad PIN,Insufficient Balance                30
Bad PIN,Technica

In [ ]:
# Impute categorical features with a placeholder string
df_merged['Errors?'].fillna('No Error', inplace=True)
df_merged['Merchant State'].fillna('Unknown', inplace=True)
df_merged['Card Brand'].fillna('Unknown', inplace=True)
df_merged['Card Type'].fillna('Unknown', inplace=True)
df_merged['Gender'].fillna('Unknown', inplace=True)

# Impute numeric features with the median
for col in ['Zip', 'Credit Limit', 'Current Age', 'Yearly Income - Person', 'FICO Score', 'Num Credit Cards']:
    if col in df_merged.columns:
        # Clean the column before converting to numeric
        if col in ['Credit Limit', 'Yearly Income - Person']:
            df_merged[col] = df_merged[col].astype(str).str.replace(r'[$,]', '', regex=True)
        df_merged[col] = pd.to_numeric(df_merged[col], errors='coerce')
        median_val = df_merged[col].median()
        df_merged[col].fillna(median_val, inplace=True)

df_merged.dropna(subset=['Expires', 'Acct Open Date', 'Card on Dark Web'], inplace=True)

# Sanity Check
print("Missing values after imputation:\n")
print(df_merged.isnull().sum())

/tmp/ipython-input-3476360498.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merged['Card Brand'].fillna('Unknown', inplace=True)
/tmp/ipython-input-3476360498.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)',

Missing values after imputation:

column00                  0
User                      0
Card                      0
Year                      0
Month                     0
Day                       0
Time                      0
Amount                    0
Use Chip                  0
Merchant Name             0
Merchant City             0
Merchant State            0
Zip                       0
MCC                       0
Errors?                   0
Is Fraud?                 0
trans_date_trans_time     0
trans_hour                0
day_of_week               0
category                  0
Card Brand                0
Card Type                 0
Expires                   0
Credit Limit              0
Acct Open Date            0
Card on Dark Web          0
Current Age               0
Gender                    0
Yearly Income - Person    0
FICO Score                0
Num Credit Cards          0
dtype: int64


## Feature Engineering for High-Cardinality Features

In [ ]:
import pandas as pd
import numpy as np

print("Starting robust feature engineering...")

if df['Amount'].dtype == 'object':
    df['Amount'] = df['Amount'].replace({r'\$': ''}, regex=True).astype(float)
    print("Converted 'Amount' column to numeric.")

df['transaction_dt'] = pd.to_datetime(df['trans_date_trans_time'], errors='coerce')
df['acct_open_dt'] = pd.to_datetime(df['Acct Open Date'], format='%m/%Y', errors='coerce')
df['expires_dt'] = pd.to_datetime(df['Expires'], format='%m/%y', errors='coerce')


df['Account_Age_Days'] = (df['transaction_dt'] - df['acct_open_dt']).dt.days
print("Created 'Account_Age_Days'.")

# Calculate the number of days until the card expires from the transaction date
df['Time_to_Expiry_Days'] = (df['expires_dt'] - df['transaction_dt']).dt.days
print("Created 'Time_to_Expiry_Days'.")


# Engineer Other High-Cardinality Features
df['Credit_Limit_Bin'] = pd.qcut(
    df['Credit Limit'],
    q=5,
    duplicates='drop',
    labels=False
)
print("Created 'Credit_Limit_Bin'.")

df['User_Frequency'] = df['User'].map(df['User'].value_counts())
print("Created 'User_Frequency'.")


# Clean Up and Drop Original/Temporary Columns
df['Account_Age_Days'].fillna(0, inplace=True)
df['Time_to_Expiry_Days'].fillna(0, inplace=True)
df['User_Frequency'].fillna(0, inplace=True)

# Define columns to drop
columns_to_drop = [
    'Unnamed: 0', 'trans_date_trans_time', 'Acct Open Date', 'Expires',
    'transaction_dt', 'acct_open_dt', 'expires_dt',
    'Credit Limit', 'User', 'Time'
]
df.drop(columns=columns_to_drop, errors='ignore', inplace=True)

print("\nAdvanced feature engineering complete.")
print("\nFinal columns in the DataFrame:")
print(df.columns.tolist())

Starting robust feature engineering...
Converted 'Amount' column to numeric.
Created 'Account_Age_Days'.
Created 'Time_to_Expiry_Days'.
Created 'Credit_Limit_Bin'.
Created 'User_Frequency'.


/tmp/ipython-input-3898486928.py:48: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Account_Age_Days'].fillna(0, inplace=True)
/tmp/ipython-input-3898486928.py:49: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us


Advanced feature engineering complete.

Final columns in the DataFrame:
['column00', 'Card', 'Year', 'Month', 'Day', 'Amount', 'Use Chip', 'Merchant Name', 'Merchant City', 'Merchant State', 'Zip', 'MCC', 'Errors?', 'Is Fraud?', 'trans_hour', 'day_of_week', 'category', 'Card Brand', 'Card Type', 'Card on Dark Web', 'Current Age', 'Gender', 'Yearly Income - Person', 'FICO Score', 'Num Credit Cards', 'Account_Age_Days', 'Time_to_Expiry_Days', 'Credit_Limit_Bin', 'User_Frequency']


In [ ]:
from pathlib import Path

# --- Save the Feature-Engineered DataFrame ---
print("\nSaving the feature-engineered DataFrame to overwrite the old CSV...")

output_csv_path = Path("/content/drive/MyDrive/Colab Notebooks/MA/datasets/archive//credit_card_transactions_ibm_v2_2008_prepped_merged.csv")

df.to_csv(output_csv_path, index=False)

print(f"Successfully saved feature-engineered data to: {output_csv_path}")


Saving the feature-engineered DataFrame to overwrite the old CSV...
Successfully saved feature-engineered data to: /content/drive/MyDrive/Colab Notebooks/MA/datasets/archive/credit_card_transactions_ibm_v2_2008_prepped_merged.csv


## Preprocessing Pipeline

### Preprocessor for kernel-based models

In [ ]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.feature_extraction import FeatureHasher
import cloudpickle
from pathlib import Path
from sklearn.model_selection import train_test_split
import numpy as np

df_actual = df.copy()
print(f"Working with DataFrame of shape: {df_actual.shape}")


print("\n--- Verifying data types before preprocessing ---")
for col in df_actual.select_dtypes(include=['object', 'category']).columns:
    temp_numeric_col = pd.to_numeric(df_actual[col], errors='coerce')
    if not temp_numeric_col.isnull().all():
        df_actual[col] = temp_numeric_col.astype(np.float32)
        print(f"  Column '{col}' confirmed/converted to numeric.")
    else:
        df_actual[col] = df_actual[col].astype('category')
        print(f"  Column '{col}' confirmed as categorical.")


y_full = df_actual["Is Fraud?"].astype(int)
X_full = df_actual.drop(columns=["Is Fraud?"])


# Stratified Sampling for Preprocessor Fitting
sample_size_for_fitting = 1_000_000
if len(X_full) > sample_size_for_fitting:
    # Use a fraction of the data to fit the preprocessor to save memory and time
    _, X_sample, _, y_sample = train_test_split(
        X_full, y_full,
        test_size=sample_size_for_fitting, # Use 1M rows for the sample
        random_state=42,
        stratify=y_full
    )
    print(f"\nUsing a stratified sample of {len(X_sample)} rows to fit the preprocessor.")
else:
    X_sample = X_full
    print("\nDataset size is manageable. Using full dataset for preprocessor fitting.")


# Dynamically identify column types
numeric_cols = X_sample.select_dtypes(include=np.number).columns.tolist()
cat_cols = X_sample.select_dtypes(include=['category', 'object']).columns.tolist()

# Differentiate between low and high cardinality categorical features
low_card_cols = [c for c in cat_cols if X_sample[c].nunique() <= 10]
high_card_cols = [c for c in cat_cols if X_sample[c].nunique() > 10]

print("\n--- Columns determined for kernel-preprocessor ---")
print(f"Numeric cols: {numeric_cols}")
print(f"Low Cardinality Cols: {low_card_cols}")
print(f"High Cardinality Cols: {high_card_cols}")


# Define Preprocessing Pipelines
numeric_pipe = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("scale", StandardScaler())
])

low_card_pipe = Pipeline([
    ("impute", SimpleImputer(strategy="constant", fill_value="missing")),
    ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=True))
])

high_card_pipe = Pipeline([
    ("impute", SimpleImputer(strategy="constant", fill_value="missing")),
    ("hash", FeatureHasher(n_features=128, input_type='string'))
])


# Combine pipelines in ColumnTransformer
preprocessor_kernel = ColumnTransformer([
    ("num", numeric_pipe, numeric_cols),
    ("lowc", low_card_pipe, low_card_cols),
    ("highc", high_card_pipe, high_card_cols),
], remainder='passthrough', sparse_threshold=0.3)


# Fit and Save the Preprocessor
print(f"\nFitting preprocessor_kernel on the sample data...")
preprocessor_kernel.fit(X_sample)
print("Preprocessor fitted successfully.")

with open('/content/drive/MyDrive/Colab Notebooks/MA/Altman_Synth/preprocessors/preprocessor_kernel.pkl', 'wb') as f:
    cloudpickle.dump(preprocessor_kernel, f)
print("\n 'preprocessor_kernel.pkl' saved successfully.")

Working with DataFrame of shape: (1223460, 29)

--- Verifying data types before preprocessing ---
  Column 'Use Chip' confirmed as categorical.
  Column 'Merchant City' confirmed as categorical.
  Column 'Merchant State' confirmed as categorical.
  Column 'Errors?' confirmed as categorical.
  Column 'day_of_week' confirmed as categorical.
  Column 'category' confirmed as categorical.
  Column 'Card Brand' confirmed as categorical.
  Column 'Card Type' confirmed as categorical.
  Column 'Gender' confirmed as categorical.

Using a stratified sample of 1000000 rows to fit the preprocessor.

--- Columns determined for kernel-preprocessor ---
Numeric cols: ['column00', 'Card', 'Year', 'Month', 'Day', 'Amount', 'Merchant Name', 'Zip', 'MCC', 'trans_hour', 'Current Age', 'Yearly Income - Person', 'FICO Score', 'Num Credit Cards', 'Account_Age_Days', 'Time_to_Expiry_Days', 'Credit_Limit_Bin', 'User_Frequency']
Low Cardinality Cols: ['Use Chip', 'day_of_week', 'Card Brand', 'Card Type', 'Gender

### Preprocessor for tree-based models

In [ ]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
import cloudpickle
from pathlib import Path

df_actual = df.copy()

y_full = df_actual["Is Fraud?"].astype(int)
X_full = df_actual.drop(columns=["Is Fraud?"])

# Dynamically identify column types
numeric_cols = X_full.select_dtypes(include=np.number).columns.tolist()
cat_cols = X_full.select_dtypes(include=['category', 'object']).columns.tolist()
low_card_cols = [c for c in cat_cols if X_full[c].nunique() <= 10]
high_card_cols = [c for c in cat_cols if X_full[c].nunique() > 10]

print("\n--- Columns determined for tree-preprocessor ---")
print(f"Numeric cols: {numeric_cols}")
print(f"Low Cardinality Cols: {low_card_cols}")
print(f"High Cardinality Cols: {high_card_cols}")


numeric_pipe_tree = Pipeline([
    ("impute", SimpleImputer(strategy="median"))
])

low_card_pipe_tree = Pipeline([
    ("impute", SimpleImputer(strategy="constant", fill_value="missing")),
    ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

high_card_pipe_tree = Pipeline([
    ("impute", SimpleImputer(strategy="constant", fill_value="missing")),
    ("ord", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1))
])


# Combine pipelines in ColumnTransformer
preprocessor_tree = ColumnTransformer([
    ("num", numeric_pipe_tree, numeric_cols),
    ("lowc", low_card_pipe_tree, low_card_cols),
    ("highc", high_card_pipe_tree, high_card_cols)
], remainder='passthrough')


# Fit and Save the Preprocessor
print(f"\nFitting preprocessor_tree on the full feature data...")
preprocessor_tree.fit(X_full)
print("Preprocessor fitted successfully.")

with open('/content/drive/MyDrive/Colab Notebooks/MA/Altman_Synth/preprocessors/preprocessor_tree.pkl', 'wb') as f:
    cloudpickle.dump(preprocessor_tree, f)
print("\n 'preprocessor_tree.pkl' saved successfully.")


--- Columns determined for tree-preprocessor ---
Numeric cols: ['column00', 'Card', 'Year', 'Month', 'Day', 'Amount', 'Merchant Name', 'Zip', 'MCC', 'trans_hour', 'Current Age', 'Yearly Income - Person', 'FICO Score', 'Num Credit Cards', 'Account_Age_Days', 'Time_to_Expiry_Days', 'Credit_Limit_Bin', 'User_Frequency']
Low Cardinality Cols: ['Use Chip', 'day_of_week', 'Card Brand', 'Card Type', 'Gender']
High Cardinality Cols: ['Merchant City', 'Merchant State', 'Errors?', 'category']

Fitting preprocessor_tree on the full feature data...
Preprocessor fitted successfully.

 'preprocessor_tree.pkl' saved successfully.


### Preprocessor for autoencoders

In [ ]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.feature_extraction import FeatureHasher
import cloudpickle
from pathlib import Path
from sklearn.model_selection import train_test_split
import numpy as np

df_actual = df.copy()

# Separate features (X) from target (y)
y_full = df_actual["Is Fraud?"].astype(int)
X_full = df_actual.drop(columns=["Is Fraud?"])

# Stratified Sampling for Preprocessor Fitting
sample_size_for_fitting = 1_000_000
if len(X_full) > sample_size_for_fitting:
    _, X_sample, _, y_sample = train_test_split(
        X_full, y_full,
        test_size=sample_size_for_fitting,
        random_state=42,
        stratify=y_full
    )
    print(f"\nUsing a stratified sample of {len(X_sample)} rows to fit the preprocessor.")
else:
    X_sample = X_full

# Dynamically identify column types
numeric_cols = X_sample.select_dtypes(include=np.number).columns.tolist()
cat_cols = X_sample.select_dtypes(include=['category', 'object']).columns.tolist()
low_card_cols = [c for c in cat_cols if X_sample[c].nunique() <= 10]
high_card_cols = [c for c in cat_cols if X_sample[c].nunique() > 10]

print("\n--- Columns determined for autoencoder-preprocessor ---")
print(f"Numeric cols: {numeric_cols}")
print(f"Low Cardinality Cols: {low_card_cols}")
print(f"High Cardinality Cols: {high_card_cols}")

# Define Preprocessing Pipelines
numeric_pipe_autoenc = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("scale", MinMaxScaler())  # MinMaxScaler
])

low_card_pipe_autoenc = Pipeline([
    ("impute", SimpleImputer(strategy="constant", fill_value="missing")),
    ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False)) # Dense output
])

high_card_pipe_autoenc = Pipeline([
    ("impute", SimpleImputer(strategy="constant", fill_value="missing")),
    ("hash", FeatureHasher(n_features=128, input_type='string'))
])

# Combine pipelines in ColumnTransformer
# sparse_threshold=0.0 forces a dense NumPy array output
preprocessor_autoenc = ColumnTransformer([
    ("num", numeric_pipe_autoenc, numeric_cols),
    ("lowc", low_card_pipe_autoenc, low_card_cols),
    ("highc", high_card_pipe_autoenc, high_card_cols),
], remainder='passthrough', sparse_threshold=0.0)

# Fit and Save the Preprocessor
print(f"\nFitting preprocessor_autoenc on the sample data...")
preprocessor_autoenc.fit(X_sample)
print("Preprocessor fitted successfully.")

with open('/content/drive/MyDrive/Colab Notebooks/MA/Altman_Synth/preprocessors/preprocessor_autoenc.pkl', 'wb') as f:
    cloudpickle.dump(preprocessor_autoenc, f)
print("\n 'preprocessor_autoenc.pkl' saved successfully.")


Using a stratified sample of 1000000 rows to fit the preprocessor.

--- Columns determined for autoencoder-preprocessor ---
Numeric cols: ['column00', 'Card', 'Year', 'Month', 'Day', 'Amount', 'Merchant Name', 'Zip', 'MCC', 'trans_hour', 'Current Age', 'Yearly Income - Person', 'FICO Score', 'Num Credit Cards', 'Account_Age_Days', 'Time_to_Expiry_Days', 'Credit_Limit_Bin', 'User_Frequency']
Low Cardinality Cols: ['Use Chip', 'day_of_week', 'Card Brand', 'Card Type', 'Gender']
High Cardinality Cols: ['Merchant City', 'Merchant State', 'Errors?', 'category']

Fitting preprocessor_autoenc on the sample data...
Preprocessor fitted successfully.

 'preprocessor_autoenc.pkl' saved successfully.


## Saving final Products to disk

In [ ]:
import pandas as pd
import cloudpickle
from pathlib import Path
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.sparse import save_npz, issparse, vstack
import gc
import warnings

# Define Paths and Constants
PREPROCESSOR_DIR = Path('/content/drive/MyDrive/Colab Notebooks/MA/Altman_Synth/preprocessors')
OUTPUT_DIR = Path('/content/drive/MyDrive/Colab Notebooks/MA/Altman_Synth/processed_data')
PREPROCESSOR_FILES = [
    "preprocessor_kernel.pkl",
    "preprocessor_autoenc.pkl",
    "preprocessor_tree.pkl"
]
TARGET_COL = "Is Fraud?"
CHUNK_SIZE = 200000

print(f"Starting with DataFrame of shape: {df.shape}")
y = df[TARGET_COL].astype(int)
X = df.drop(columns=[TARGET_COL])

del df # Free up memory
gc.collect()
print("Original DataFrame deleted from memory.")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

del X, y # Free up more memory
gc.collect()
print("Pre-split X and y deleted from memory.")


# Save Labels
np.save(OUTPUT_DIR / "y_train.npy", y_train.to_numpy())
np.save(OUTPUT_DIR / "y_test.npy", y_test.to_numpy())
print(f"\nSaved y_train.npy and y_test.npy to: {OUTPUT_DIR}")


# Process and Save Data for each Preprocessor
for pkl_file in PREPROCESSOR_FILES:
    preprocessor_path = PREPROCESSOR_DIR / pkl_file
    print(f"\n--- Processing for: {preprocessor_path.name} ---")

    if not preprocessor_path.exists():
        print(f"WARNING: Skipping {preprocessor_path.name}, file not found.")
        continue

    with open(preprocessor_path, "rb") as f:
        preprocessor = cloudpickle.load(f)

    base_name = preprocessor_path.stem.replace('preprocessor_', '')

    # Process and save X_train
    print(f"Processing X_train in chunks...")
    train_chunks = [preprocessor.transform(X_train.iloc[i:i + CHUNK_SIZE]) for i in range(0, len(X_train), CHUNK_SIZE)]
    X_train_transformed = vstack(train_chunks) if issparse(train_chunks[0]) else np.vstack(train_chunks)

    if issparse(X_train_transformed):
        save_npz(OUTPUT_DIR / f"X_train_transformed_{base_name}.npz", X_train_transformed)
    else:
        np.save(OUTPUT_DIR / f"X_train_transformed_{base_name}.npy", X_train_transformed)
    print(f"Saved transformed X_train for '{base_name}'.")
    del train_chunks, X_train_transformed
    gc.collect()

    # Process and save X_test
    print(f"Processing X_test in chunks...")
    test_chunks = [preprocessor.transform(X_test.iloc[i:i + CHUNK_SIZE]) for i in range(0, len(X_test), CHUNK_SIZE)]
    X_test_transformed = vstack(test_chunks) if issparse(test_chunks[0]) else np.vstack(test_chunks)

    if issparse(X_test_transformed):
        save_npz(OUTPUT_DIR / f"X_test_transformed_{base_name}.npz", X_test_transformed)
    else:
        np.save(OUTPUT_DIR / f"X_test_transformed_{base_name}.npy", X_test_transformed)
    print(f"Saved transformed X_test for '{base_name}'.")
    del test_chunks, X_test_transformed
    gc.collect()

print("\nData preparation complete. All final datasets have been saved successfully.")

Starting with DataFrame of shape: (1223460, 29)
Original DataFrame deleted from memory.
Pre-split X and y deleted from memory.

Saved y_train.npy and y_test.npy to: /content/drive/MyDrive/Colab Notebooks/MA/Altman_Synth/processed_data

--- Processing for: preprocessor_kernel.pkl ---
Processing X_train in chunks...


/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instanc

Saved transformed X_train for 'kernel'.
Processing X_test in chunks...


/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


Saved transformed X_test for 'kernel'.

--- Processing for: preprocessor_autoenc.pkl ---
Processing X_train in chunks...


/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instanc

Saved transformed X_train for 'autoenc'.
Processing X_test in chunks...


/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


Saved transformed X_test for 'autoenc'.

--- Processing for: preprocessor_tree.pkl ---
Processing X_train in chunks...
Saved transformed X_train for 'tree'.
Processing X_test in chunks...
Saved transformed X_test for 'tree'.

Data preparation complete. All final datasets have been saved successfully.
